# Лабораторная работа 2
## Генералов Даниил, 1032212280

> В данной лабораторной работе предлагается познакомится с API реального веб-сервиса — random.org. Этот сервис предназначен для получения последовательности случайных чисел. API реализован по протоколу JSON-RPC
> 
> Для выполнения задания желательно зарегистрировать двух-трех пользователей на всю группу. Для совершения запросов в дальнейшем потребуется только API ключ.


> Исходный код этого файла доступен в git-репозитории: https://github.com/danya02/rudn-year3-api-labs/blob/main/lab2/main.ipynb

## Ключ авторизации

Для доступа к API сайта random.org требуется API-ключ.
Для того, чтобы получить API-ключи, традиционно требуется использовать веб-интерфейс,
чтобы зарегестрироваться, зайти в панель управления,
и затем создать ключ.

Такие ключи имеют действие, похожее на пароли, поэтому их следует хранить в тайне.
Но это сложно сделать, когда исходный код этой лабораторной работы находится в публичном доступе.
Поэтому большинство сервисов также имеют опцию отозвать выданный API-ключ, чтобы он больше не работал --
я сделал это, прежде чем публиковать этот документ.

Использованный ключ написан в переменной ниже. Если вы хотите запустить этот код сами, то следует указать свой ключ.


In [2]:
API_KEY = "165cbea2-fca9-4393-b8c9-fd7758e84a02"  # Если вы читаете это, ключ уже не работает

## JSON-RPC

Один из RPC-протоколов -- это JSON-RPC. Если API работает через JSON-RPC, то пользователь может отправлять команды выполнить какую-то функцию (*remote procedure*), указать для нее аргументы, и получить ответ в стандартном формате.

По спецификации JSON-RPC, клиент должен отправить объект, содержащий следующие поля:

- `jsonrpc` -- версия протокола, всегда равно "2.0"
- `method` -- название метода, который следует вызвать
- `params` -- значения аргументов к методу
- `id` -- идентификатор запроса

В ответ на это сервер отправит объект с следующими полями:

- `jsonrpc` -- так же как в запросе
- `result` -- если успешно выполнено, результат выполнения метода
- `error` -- если была ошибка, описание ошибки
- `id` -- то же самое значение, как и `id` в запросе. Клиент может использовать это для корреляции запросов и ответов.

## generateIntegers

Для того, чтобы выполнить метод `generateIntegers`, нужно отправить POST-запрос на определенный URL (фиксированный для всех методов). Для этого я использую библиотеку [requests](https://docs.python-requests.org/en/latest/index.html)

In [4]:
import requests
BASE_URL = 'https://api.random.org/json-rpc/4/invoke'

resp = requests.post(BASE_URL, json={
    "jsonrpc": "2.0",
    "method": "generateIntegers",
    "id": 69420,
    "params": {
        "apiKey": API_KEY,
        "n": 5,
        "min": -127,
        "max": 128
    }
})

После выполнения запроса можно посмотреть на его содержимое, а также на возвращаемые данные:

In [15]:
import pprint
print("Header в запросе:")
for k, v in resp.request.headers.items():
    print(f'  - {k}: {v}')
print()

print("Header в ответе:")
for k, v in resp.headers.items():
    print(f'  - {k}: {v}')

print()
print("Код ответа:", resp.status_code)
print()
print("JSON ответа:")
pprint.pprint(resp.json())

Header в запросе:
  - User-Agent: python-requests/2.31.0
  - Accept-Encoding: gzip, deflate, br
  - Accept: */*
  - Connection: keep-alive
  - Content-Length: 156
  - Content-Type: application/json

Header в ответе:
  - Date: Fri, 19 Apr 2024 19:27:08 GMT
  - Content-Type: application/json; charset=utf-8
  - Transfer-Encoding: chunked
  - Connection: keep-alive
  - X-Content-Type-Options: nosniff
  - Access-Control-Allow-Origin: *
  - Access-Control-Allow-Headers: Origin, X-Requested-With, Content-Type, Accept
  - CF-Cache-Status: DYNAMIC
  - Set-Cookie: __cflb=02DiuEJLVpaAQLxvFoC2iH8cD76FLs8dnz8s4UDCVP6XM; SameSite=Lax; path=/; expires=Fri, 26-Apr-24 19:27:08 GMT; HttpOnly
  - Strict-Transport-Security: max-age=31536000; includeSubDomains; preload
  - Server: cloudflare
  - CF-RAY: 876f4a4dade59da3-DME
  - Content-Encoding: br

Код ответа: 200

JSON ответа:
{'id': 69420,
 'jsonrpc': '2.0',
 'result': {'advisoryDelay': 2170,
            'bitsLeft': 249960,
            'bitsUsed': 40,
 

В ответе сервера присутствуют следующие поля:

- `id`: тот же самый `id`, как был указан в запросе
- `jsonrpc`: версия протокола, 2.0
- `result`: результат успешного выполнения метода:
  - `advisoryDelay`: сколько времени сервер рекомендует ждать прежде чем делать следующий запрос, чтобы не превысить rate limit
  - `bitsLeft`: сколько битов случайности осталось в квоте этого ключа
  - `bitsUsed`: сколько битов случайности требовалось, чтобы выполнить этот запрос
  - `requestsLeft`: сколько запросов еще можно сделать с этим ключом
  - `random`: информация о случайных числах:
    - `data`: сгенерированные числа
    - `completionTime`: время, когда был осуществлен запрос

В header ответа есть много различных:

  - `Date`: в какое время на сервере произошел запрос
  - `Content-Type`: сервер отправил 
  - `Content-Encoding`: каким алгоритмом было обработано тело ответа (какой алгоритм сжатия)
  - `Transfer-Encoding`: по проводу содержимое было передано по кускам, где каждый кусок имеет в начале свою длину
  - `Connection`: если клиент хочет делать еще запросы к этому серверу, их можно отправлять через то же самое TCP-соединение
  - `Strict-Transport-Security`: этот сайт гарантирует, что он будет доступен по HTTPS как минимум год, в том числе его поддомены, и этот факт можно записать в перманентную базу данных браузера
  - `X-Content-Type-Options`: браузеру не стоит пытаться догадаться, какой тип данных в этом документе -- содержимое `Content-Type` задано правильно
  - `Access-Control-Allow-Origin`: всем сайтам разрешается посылать сюда запросы...
  - `Access-Control-Allow-Headers`: ...и если эти запросы содержат эти headers, то их разрешается передавать данному серверу
  - `Server`: сайт использует прокси от Cloudflare
  - `CF-Cache-Status`: содержимое этой страницы не предполагалось к кешированию в Cloudflare CDN
  - `CF-RAY`: этот идентификатор можно использовать, чтобы отследить этот запрос в логах Cloudflare
  - `Set-Cookie`: браузеру стоит запомнить это значение и передавать его с последующими запросами как cookie

Можно заказать случайные числа в другой системе исчисления, например восьмеричной:

In [17]:
resp2 = requests.post(BASE_URL, json={
    "jsonrpc": "2.0",
    "method": "generateIntegers",
    "id": 69420,
    "params": {
        "apiKey": API_KEY,
        "n": 5,
        "min": -127,
        "max": 128,
        "base": 8
    }
})

Тогда ответы возвращаются в виде строк, которые содержат числа в нужной системе, а не JSON-тип Number:

In [18]:
pprint.pprint(resp2.json())

{'id': 69420,
 'jsonrpc': '2.0',
 'result': {'advisoryDelay': 2700,
            'bitsLeft': 249920,
            'bitsUsed': 40,
            'random': {'completionTime': '2024-04-19 19:56:40Z',
                       'data': ['055', '122', '045', '157', '163']},
            'requestsLeft': 998}}


## Ошибки

Если метод в JSON-RPC не может вернуть результат, то он возвращает ошибку. В ошибке есть код ошибки, строка с человеко-читаемым объяснением ошибки, а также данные для форматирования этой строки на другом языке.

В зависимости от ситуации могут быть разные типы ошибок:

In [22]:
err1 = requests.post(BASE_URL, json={
    "jsonrpc": "2.0",
    "method": "generateIntegers",
    "id": 69420,
    "params": {
        "apiKey": API_KEY,
        "n": 15,  # Слишком много чисел запрошено
        "min": 0,
        "max": 10,
        "replacement": False
    }
})

In [23]:
pprint.pprint(err1.json())

{'error': {'code': 301,
           'data': [15, 11],
           'message': 'You requested 15 values without replacement but the '
                      'domain you specified only contains 11'},
 'id': 69420,
 'jsonrpc': '2.0'}


In [27]:
err2 = requests.post(BASE_URL, data="этот json поддельный".encode(), headers={'content-type': 'application/json'})

In [28]:
pprint.pprint(err2.json())

{'error': {'code': -32700, 'data': None, 'message': 'Parse error'},
 'id': None,
 'jsonrpc': '2.0'}


## generateIntegerSequences

Этот метод позволяет запросить несколько числовых последовательностей вместе, в отличии от `generateIntegers`, который генерирует только одну. Это позволяет сэкономить HTTP-запросы, когда известно, что требуется несколько последовательностей одновременно: квота на количество запросов гораздо ниже, чем квота на количество случайных битов.

In [29]:
resp3 = requests.post(BASE_URL, json={
    "jsonrpc": "2.0",
    "method": "generateIntegerSequences",
    "id": 69420,
    "params": {
        "apiKey": API_KEY,
        "n": 5,
        "length": 10,
        "min": -64,
        "max": 63
    }
})

In [30]:
print("Header в запросе:")
for k, v in resp3.request.headers.items():
    print(f'  - {k}: {v}')
print()

print("Header в ответе:")
for k, v in resp3.headers.items():
    print(f'  - {k}: {v}')

print()
print("Код ответа:", resp3.status_code)
print()
print("JSON ответа:")
pprint.pprint(resp3.json())

Header в запросе:
  - User-Agent: python-requests/2.31.0
  - Accept-Encoding: gzip, deflate, br
  - Accept: */*
  - Connection: keep-alive
  - Content-Length: 176
  - Content-Type: application/json

Header в ответе:
  - Date: Fri, 19 Apr 2024 20:27:49 GMT
  - Content-Type: application/json; charset=utf-8
  - Transfer-Encoding: chunked
  - Connection: keep-alive
  - X-Content-Type-Options: nosniff
  - Access-Control-Allow-Origin: *
  - Access-Control-Allow-Headers: Origin, X-Requested-With, Content-Type, Accept
  - CF-Cache-Status: DYNAMIC
  - Set-Cookie: __cflb=02DiuEJLVpaAQLxvFoC2iH8cD76FLs8doiJFNfgn4awwR; SameSite=Lax; path=/; expires=Fri, 26-Apr-24 20:27:49 GMT; HttpOnly
  - Strict-Transport-Security: max-age=31536000; includeSubDomains; preload
  - Server: cloudflare
  - CF-RAY: 876fa3344f409dc1-DME
  - Content-Encoding: br

Код ответа: 200

JSON ответа:
{'id': 69420,
 'jsonrpc': '2.0',
 'result': {'advisoryDelay': 1930,
            'bitsLeft': 249530,
            'bitsUsed': 350,


Разницы в HTTP-header здесь нет: при использовании JSON-RPC значения header в целом можно игнорировать.
Однако в этом методе отличается формат `result.random.data`: теперь здесь возвращается список из списков случайных чисел, а не просто один список. Это соответствует искомым последовательностям чисел.

## Использование

Чтобы симулировать подброс монеты или кубика, можно использовать любой из этих двух методов, где границы интервалов соответствуют используемому инструменту.

In [31]:
resp4 = requests.post(BASE_URL, json={
    "jsonrpc": "2.0",
    "method": "generateIntegers",
    "id": 69420,
    "params": {
        "apiKey": API_KEY,
        "n": 1,
        "min": 0,
        "max": 1,
    }
})

resp5 = requests.post(BASE_URL, json={
    "jsonrpc": "2.0",
    "method": "generateIntegers",
    "id": 69420,
    "params": {
        "apiKey": API_KEY,
        "n": 1,
        "min": 1,
        "max": 6,
    }
})

In [33]:
print("Монета:", "ОРЕЛ" if resp4.json()['result']['random']['data'][0] == 1 else "РЕШКА")
print("Кубик:", resp5.json()['result']['random']['data'][0])


Монета: ОРЕЛ
Кубик: 5


Чтобы сгенерировать 5 IP-адресов (IPv4), в первом приближении можно собрать 20 чисел от 0 до 255 и объединить их в группы по четыре.

Это не совсем правильный способ, потому что в IPv4-пространстве есть зарезервированные адреса, которые имеют нестандартное поведение (например, все адреса между 127.0.0.0 и 127.255.255.255 относятся к одному и тому же компьютеру).

In [34]:
resp6 = requests.post(BASE_URL, json={
    "jsonrpc": "2.0",
    "method": "generateIntegerSequences",
    "id": 69420,
    "params": {
        "apiKey": API_KEY,
        "n": 5,
        "length": 4,
        "min": 0,
        "max": 255
    }
})

In [35]:
pprint.pprint(resp6.json())

{'id': 69420,
 'jsonrpc': '2.0',
 'result': {'advisoryDelay': 1980,
            'bitsLeft': 249366,
            'bitsUsed': 160,
            'random': {'completionTime': '2024-04-19 20:45:25Z',
                       'data': [[179, 164, 35, 239],
                                [172, 92, 28, 122],
                                [237, 155, 129, 250],
                                [233, 25, 184, 150],
                                [26, 165, 18, 62]]},
            'requestsLeft': 993}}


In [36]:
print("IP-адреса (?):")
for row in resp6.json()['result']['random']['data']:
    print('  -', '.'.join(map(str, row)))

IP-адреса (?):
  - 179.164.35.239
  - 172.92.28.122
  - 237.155.129.250
  - 233.25.184.150
  - 26.165.18.62


## Заключение
В этой лабораторной работе мы рассмотрели, как работать с веб-сервисом с настоящим API.
Это поможет нам выполнять следующую лабораторную работу, где мы рассмотрим, как работать с еще более сложным API Вконтакте.